In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.svm import SVR
from pprint import pprint

df_2012_train = pd.read_csv('df_2012_train.csv') 
df_2012_train = df_2012_train.set_index('fips')
df_2012_test = pd.read_csv('df_2012_test.csv') 
df_2012_test = df_2012_test.set_index('fips')
df_2016_train = pd.read_csv('df_2016_train.csv') 
df_2016_train = df_2016_train.set_index('county_name')
df_2016_test = pd.read_csv('df_2016_test.csv') 
df_2016_test = df_2016_test.set_index('county_name')

In [2]:
df_2012_train.head(2)

,state_abbreviation,county_name,diff_2012,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,perc_2races,...,hc_value_1k,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
50011,VT,Franklin County,0.235485,0.607936,0.554590,0.852168,0.752126,0.454698,0.679059,0.334973,...,0.618064,0.598671,0.573469,0.538418,0.619473,0.426309,0.575188,0.366486,0.414746,0.166732
46093,SD,Meade County,-0.428453,0.800869,0.501383,0.814002,0.806294,0.452165,0.683144,0.451818,...,0.146813,0.044439,0.491513,0.502540,0.659653,0.397049,0.523883,0.330485,0.352493,0.177449


In [3]:
df_2012_train_x = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y = df_2012_train[['diff_2012']]

df_2016_train_x = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y = df_2016_train[['diff_2016']]

df_2012_test_x = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y = df_2012_test[['diff_2012']]

df_2016_test_x = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y = df_2016_test[['diff_2016']]

full_train_x = [df_2012_train_x, df_2016_train_x]
full_train_x = pd.concat(full_train_x)
df_2012_train_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y = [df_2012_train_y, df_2016_train_y]
full_train_y = pd.concat(full_train_y)

full_test_x = [df_2012_test_x, df_2016_test_x]
full_test_x = pd.concat(full_test_x)
df_2012_test_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y = [df_2012_test_y, df_2016_test_y]
full_test_y = pd.concat(full_test_y)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
#2012 FEATURE SELECTION

from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel

rgr = SVR(kernel='linear', gamma=0.1, C=1.0)

featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2012_train_x, df_2012_train_y)

selectedFeatures = featureSelection.transform(df_2012_train_x)
#selectedFeatures

fs = df_2012_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2012_train_x = df_2012_train_x[keep]
df_2012_test_x = df_2012_test_x[keep]


ft_2012 = keep

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_hs', 'perc_f_emp_2065', 'hc_value_1k', 'perc_pov', 'num_establishments']


In [5]:
#2016 feature selection

rgr = SVR(kernel='linear', gamma=0.1, C=1.0)
featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2016_train_x, df_2016_train_y)

selectedFeatures = featureSelection.transform(df_2016_train_x)
#selectedFeatures

fs = df_2016_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2016_train_x = df_2016_train_x[keep]
df_2016_test_x = df_2016_test_x[keep]

ft_2016 = keep

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_hs', 'perc_f_emp_2065', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments', 'annual_payroll']


In [6]:
rgr = SVR(kernel='linear', gamma=0.1, C=1.0)
featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(full_train_x, full_train_y)

selectedFeatures = featureSelection.transform(full_train_x)
#selectedFeatures

fs = full_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

full_train_x = full_train_x[keep]
full_test_x = full_test_x[keep]


ft_full = keep

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_white', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_f_lang', 'perc_hs', 'perc_f_emp_2065', 'hc_value_1k', 'median_income', 'perc_pov', 'num_establishments']


In [7]:
sv = SVR()
pprint(sv.get_params())

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'auto_deprecated',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [8]:
from sklearn.model_selection import RandomizedSearchCV

# Not doing

C = [1, 5, 10]
# Number of trees in  forest
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
# Maximum number of levels in tree
gamma = ['auto', 'scale']


In [9]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'C': C,
    'kernel': kernel,
    'gamma': gamma}
# Create a based model
sv = SVR()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = sv, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [10]:
# Fit the grid search to the data
grid_search.fit(df_2012_train_x, df_2012_train_y)
grid_search.best_params_



Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  2.8min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("mse:", mse)
    print("rmse:",rmse)
    mae = mean_absolute_error(test_labels, predictions)
    print("mae:", mae)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

In [12]:
df_2012_train_x2 = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y2 = df_2012_train[['diff_2012']]

df_2016_train_x2 = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y2 = df_2016_train[['diff_2016']]

df_2012_test_x2 = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y2 = df_2012_test[['diff_2012']]

df_2016_test_x2 = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y2 = df_2016_test[['diff_2016']]

full_train_x2 = [df_2012_train_x2, df_2016_train_x2]
full_train_x2 = pd.concat(full_train_x2)
df_2012_train_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y2 = [df_2012_train_y2, df_2016_train_y2]
full_train_y2 = pd.concat(full_train_y2)

full_test_x2 = [df_2012_test_x2, df_2016_test_x2]
full_test_x2 = pd.concat(full_test_x2)
df_2012_test_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y2 = [df_2012_test_y2, df_2016_test_y2]
full_test_y2 = pd.concat(full_test_y2)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [14]:
#fts_2012 = ['population', 'perc_female', 'perc_o18', 'perc_o65', 'perc_hisp', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_nm', 'avg_hh_size', 'perc_1yr_res', 'perc_hs', 'perc_f_emp_2065', 'hc_value_1k', 'perc_pov', 'num_establishments']
test_x = df_2016_test_x2[ft_2012]
full_x = full_test_x2[ft_2012]



print('2012 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, df_2012_test_x, df_2012_test_y)
print('2012 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test_x, df_2016_test_y2)
print('2012 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2012 GRID Search on 2012
mse: 0.02239776664449778
rmse: 0.14965883416790932
mae: 0.11612799132775192
r2: 0.7335528817075143
2012 GRID Search on 2016
mse: 0.48034690629353527
rmse: 0.6930706358615515
mae: 0.6296677385167764
r2: -0.20229566992996317
2012 GRID Search on Full
mse: 0.2513723364690165
rmse: 0.5013704583130287
mae: 0.3728978649222642
r2: -0.7512253202765786


In [15]:
grid_search.best_params_

{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}

In [19]:
list(zip(ft_2012,list(grid_search.best_estimator_.support_vectors_)))

[('population',
  array([0.5545904 , 0.85216758, 0.45469769, 0.67905885, 0.18110058,
         0.98547207, 0.53616294, 0.91200021, 0.42653484, 0.82807237,
         0.8534071 , 0.84237958, 0.61806354, 0.61947295, 0.42630912])),
 ('perc_female',
  array([0.50138325, 0.81400228, 0.45216465, 0.68314424, 0.30679285,
         0.97330405, 0.4497778 , 0.87836749, 0.39004311, 0.77628861,
         0.91369011, 0.81279372, 0.14681302, 0.65965255, 0.39704933])),
 ('perc_o18',
  array([0.36387319, 0.77958381, 0.62421151, 0.81097465, 0.95468436,
         0.57447172, 0.34010891, 0.80263052, 0.49278684, 0.82329695,
         0.83065823, 0.7232583 , 0.89145605, 0.73095968, 0.19398133])),
 ('perc_o65',
  array([0.47186979, 0.87541988, 0.51941393, 0.76004305, 0.46786063,
         0.96833461, 0.27488993, 0.87220725, 0.36211626, 0.83915305,
         0.81062594, 0.81823399, 0.59131919, 0.70278843, 0.34708033])),
 ('perc_hisp',
  array([0.48587448, 0.83561916, 0.56058245, 0.76899485, 0.20775817,
         0.9860

In [20]:
list(zip(ft_2012,list(grid_search.best_estimator_.support_)))

[('population', 0),
 ('perc_female', 1),
 ('perc_o18', 2),
 ('perc_o65', 3),
 ('perc_hisp', 4),
 ('perc_white_not_hisp', 7),
 ('median_oo_value', 13),
 ('monthly_costs_nm', 17),
 ('avg_hh_size', 18),
 ('perc_1yr_res', 20),
 ('perc_hs', 22),
 ('perc_f_emp_2065', 24),
 ('hc_value_1k', 26),
 ('perc_pov', 27),
 ('num_establishments', 29)]

In [21]:
# Fit the grid search to the data
grid_search.fit(df_2016_train_x, df_2016_train_y)
grid_search.best_params_


Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  3.0min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}

In [22]:
test_x = df_2012_test_x2[ft_2016]
full_x = full_test_x2[ft_2016]

print('2016 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('2016 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, df_2016_test_x, df_2016_test_y)
print('2016 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2016 GRID Search on 2012
mse: 0.07403676993958022
rmse: 0.2720969862743434
mae: 0.22518794275942985
r2: 0.4499026999819764
2016 GRID Search on 2016
mse: 0.015210215384176443
rmse: 0.12332970195446206
mae: 0.09259341103406071
r2: 0.8343036770919616
2016 GRID Search on Full
mse: 0.04462349266187833
rmse: 0.21124273398599613
mae: 0.15889067689674527
r2: 0.5843747030731308


In [23]:
grid_search.best_params_

{'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}

In [25]:
list(zip(ft_2012,list(grid_search.best_estimator_.support_vectors_)))

[('population',
  array([0.47657801, 0.71987279, 0.5086851 , 0.48802818, 0.95408811,
         0.34971316, 0.9694968 , 0.48551988, 0.47985743, 0.33626236,
         0.91472066, 0.79810454, 0.14681302, 0.53860308, 0.52552773,
         0.3950203 , 0.49439704])),
 ('perc_female',
  array([0.36845642, 0.76674114, 0.31791999, 0.48802818, 0.92795114,
         0.75643519, 0.8891314 , 0.40756347, 0.28125579, 0.60999505,
         0.69239648, 0.61111035, 0.7472827 , 0.45309402, 0.5935187 ,
         0.28417798, 0.41757787])),
 ('perc_o18',
  array([0.64638116, 0.78955125, 0.66029563, 0.56809229, 0.93527688,
         0.51092799, 0.94617281, 0.57992171, 0.70358602, 0.34661661,
         0.87666362, 0.77966466, 0.11522505, 0.6010785 , 0.57719154,
         0.54082719, 0.63191381])),
 ('perc_o65',
  array([0.49546199, 0.80205123, 0.46544472, 0.51738625, 0.97974644,
         0.29572524, 0.98268059, 0.49407164, 0.5329894 , 0.40251027,
         0.9461034 , 0.85958772, 0.39592388, 0.61950584, 0.47744653,
   

In [26]:
list(zip(ft_2012,list(grid_search.best_estimator_.support_)))

[('population', 1),
 ('perc_female', 3),
 ('perc_o18', 4),
 ('perc_o65', 5),
 ('perc_hisp', 7),
 ('perc_white_not_hisp', 12),
 ('median_oo_value', 15),
 ('monthly_costs_nm', 18),
 ('avg_hh_size', 23),
 ('perc_1yr_res', 24),
 ('perc_hs', 26),
 ('perc_f_emp_2065', 31),
 ('hc_value_1k', 32),
 ('perc_pov', 34),
 ('num_establishments', 37)]

In [27]:
# Fit the grid search to the data
grid_search.fit(full_train_x, full_train_y)
grid_search.best_params_

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   58.1s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 10.3min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

In [28]:
test_x = df_2012_test_x2[ft_full]
test2_x = df_2016_test_x2[ft_full]

print('FULL GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('FULL GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test2_x, df_2016_test_y2)
print('FULL GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_test_x, full_test_y)

FULL GRID Search on 2012
mse: 0.021327430689977497
rmse: 0.14603914095192938
mae: 0.11160616389718797
r2: 0.746294539772163
FULL GRID Search on 2016
mse: 0.014935927578621472
rmse: 0.12221263264745372
mae: 0.09156784254027239
r2: 0.8372905046906548
FULL GRID Search on Full
mse: 0.018131679134299482
rmse: 0.13465392357558498
mae: 0.10158700321873018
r2: 0.8001334770552501


In [29]:
list(zip(ft_2012,list(grid_search.best_estimator_.support_vectors_)))

[('population',
  array([0.5545904 , 0.85216758, 0.45469769, 0.67905885, 0.97868798,
         0.18110058, 0.98547207, 0.53616294, 0.91200021, 0.42653484,
         0.82807237, 0.37273011, 0.8534071 , 0.84237958, 0.61806354,
         0.57346878, 0.61947295, 0.42630912])),
 ('perc_female',
  array([0.50138325, 0.81400228, 0.45216465, 0.68314424, 0.95889245,
         0.30679285, 0.97330405, 0.4497778 , 0.87836749, 0.39004311,
         0.77628861, 0.29188776, 0.91369011, 0.81279372, 0.14681302,
         0.49151262, 0.65965255, 0.39704933])),
 ('perc_o18',
  array([0.36387319, 0.77958381, 0.62421151, 0.81097465, 0.84882874,
         0.95468436, 0.57447172, 0.34010891, 0.80263052, 0.49278684,
         0.82329695, 0.88816141, 0.83065823, 0.7232583 , 0.89145605,
         0.39459767, 0.73095968, 0.19398133])),
 ('perc_o65',
  array([0.47186979, 0.87541988, 0.51941393, 0.76004305, 0.95939769,
         0.46786063, 0.96833461, 0.27488993, 0.87220725, 0.36211626,
         0.83915305, 0.52026954, 0.8

In [30]:
list(zip(ft_2012,list(grid_search.best_estimator_.support_)))

[('population', 0),
 ('perc_female', 1),
 ('perc_o18', 2),
 ('perc_o65', 3),
 ('perc_hisp', 4),
 ('perc_white_not_hisp', 6),
 ('median_oo_value', 7),
 ('monthly_costs_nm', 9),
 ('avg_hh_size', 13),
 ('perc_1yr_res', 16),
 ('perc_hs', 17),
 ('perc_f_emp_2065', 20),
 ('hc_value_1k', 22),
 ('perc_pov', 24),
 ('num_establishments', 26)]